### HW1： 

In [ ]:

path = 'C:/Users/mengh/Dropbox/FRE-7871-News-Analytics-and-Machine-Learning/HW1/economist/'
file_names = os.listdir(path)

print(file_names)
# Create Dictionary for File Name and Text
file_name_and_text = {}
for file in file_names:
    with open(path + file, "r") as target_file:
         file_name_and_text[file] = target_file.read()
file_data = (pd.DataFrame.from_dict(file_name_and_text, orient='index')
             .reset_index().rename(index = str, columns = {'index': 'file_name', 0: 'text'}))

file_data.head()

In [26]:
import pandas as pd
import os

path = 'C:/Users/mengh/Dropbox/FRE-7871-News-Analytics-and-Machine-Learning/HW1/economist/'
sub_path_names = os.listdir(path)

# Create Dictionary for File Name and Text
file_name_and_text = {}
for sub_path in sub_path_names:
    file_names = os.listdir(path+sub_path)
    for file in file_names:
        if file.endswith(".txt"):
            with open(path + sub_path + '/'+ file, "r") as target_file:
                 file_name_and_text[file] = target_file.read()

file_data = (pd.DataFrame.from_dict(file_name_and_text, orient='index')
             .reset_index().rename(index = str, columns = {'index': 'file_name', 0: 'text'}))

file_data.head(200)

,file_name,text
0,africa.1.txt,\n ...
1,africa.10.txt,\n ...
2,africa.11.txt,\n The Kurds say they still want more of it ...
3,africa.12.txt,\n A politician who paid the price for telli...
4,africa.13.txt,\n President Mwai Kibaki is loth to back dow...
5,africa.14.txt,"\n Jacob Zuma, presidential hopeful\n\n ..."
6,africa.15.txt,\n Syria is finding it hard to cope with the...
7,africa.16.txt,\n ...
8,africa.17.txt,\n Iraq's Kurdish soldiers have been welcome...
9,africa.18.txt,\n Jobs and corruption are the issues in a c...


In [25]:
print(file_data.loc['0', 'text'])


                                                                  Reuters

                                  Reuters

   Are the latest terrorist bombs in north Africa home-grown, inspired by
   al-Qaeda or actually organised by it?


   THE bombs that shook Algiers and ripped the façade off the prime
   minister's office this week, killing some 30 people, suggest a
   worrying resurgence of the country's Islamist militants since they
   rebranded themselves as al-Qaeda's official arm in the Maghreb. This
   week's counter-attack on bombers in Morocco raises similar questions.
   But they were probably different sorts of terrorists. There is no
   presumption yet that al-Qaeda has effectively activated a joint north
   African front, though it would surely like to do so.

   It is the first time in years that Algeria's capital has been attacked
   on such a scale. Earlier this year there were terrorist shootings in
   Tunisia. In Morocco this week four suicide bombers died in Casablan

In [34]:
import re

file_1 = re.sub('[^a-zA-Z]', ' ', file_data['text'][0])
print(file_1)


                                                                   Reuters                                    Reuters     Are the latest terrorist bombs in north Africa home grown  inspired by    al Qaeda or actually organised by it       THE bombs that shook Algiers and ripped the fa ade off the prime    minister s office this week  killing some    people  suggest a    worrying resurgence of the country s Islamist militants since they    rebranded themselves as al Qaeda s official arm in the Maghreb  This    week s counter attack on bombers in Morocco raises similar questions     But they were probably different sorts of terrorists  There is no    presumption yet that al Qaeda has effectively activated a joint north    African front  though it would surely like to do so      It is the first time in years that Algeria s capital has been attacked    on such a scale  Earlier this year there were terrorist shootings in    Tunisia  In Morocco this week four suicide bombers died in Casablan

In [35]:
file_1 = file_1.lower()
file_1 = file_1.split()

In [33]:
file_1

['Reuters',
 'Reuters',
 'Are',
 'the',
 'latest',
 'terrorist',
 'bombs',
 'in',
 'north',
 'Africa',
 'home',
 'grown',
 'inspired',
 'by',
 'al',
 'Qaeda',
 'or',
 'actually',
 'organised',
 'by',
 'it',
 'THE',
 'bombs',
 'that',
 'shook',
 'Algiers',
 'and',
 'ripped',
 'the',
 'fa',
 'ade',
 'off',
 'the',
 'prime',
 'minister',
 's',
 'office',
 'this',
 'week',
 'killing',
 'some',
 'people',
 'suggest',
 'a',
 'worrying',
 'resurgence',
 'of',
 'the',
 'country',
 's',
 'Islamist',
 'militants',
 'since',
 'they',
 'rebranded',
 'themselves',
 'as',
 'al',
 'Qaeda',
 's',
 'official',
 'arm',
 'in',
 'the',
 'Maghreb',
 'This',
 'week',
 's',
 'counter',
 'attack',
 'on',
 'bombers',
 'in',
 'Morocco',
 'raises',
 'similar',
 'questions',
 'But',
 'they',
 'were',
 'probably',
 'different',
 'sorts',
 'of',
 'terrorists',
 'There',
 'is',
 'no',
 'presumption',
 'yet',
 'that',
 'al',
 'Qaeda',
 'has',
 'effectively',
 'activated',
 'a',
 'joint',
 'north',
 'African',
 'front

In [38]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords # 虚词字典
file_1 = [ps.stem(word) for word in file_1 if not word in set(stopwords.words('english'))]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mengh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'ps' is not defined